In [1]:
import numpy as np

import typing
from typing import Any, Tuple

import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import tensorflow_text as tf_text

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


In [2]:
path="data/ita.txt"

# Load the dataset and separate input from target

In [15]:
def load_data(path):
    """
    load the files and return coupled list of input target
    """
    with open(path, "r",encoding="utf-8") as f:
        text =f.read() 

    lines = text.splitlines()
    pairs = [line.split('\t') for line in lines]

    inp = [inp for targ, inp,attr in pairs]
    targ = [targ for targ, inp,attr in pairs]

    return targ, inp

In [18]:
targ, inp = load_data(path)

In [19]:
BUFFER_SIZE = len(inp)
BATCH_SIZE = 64

dataset = tf.data.Dataset.from_tensor_slices((inp, targ)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)

# Now we want to include all preprocessing inside the model 

In order to be able to export it as tf_saved model

In [20]:
def tf_lower_and_split_punct(text):
    # Split accecented characters.
    text = tf_text.normalize_utf8(text, 'NFKD')
    text = tf.strings.lower(text)
    # Keep space, a to z, and select punctuation.
    text = tf.strings.regex_replace(text, '[^ a-z.?!,]', '')
    # Add spaces around punctuation.
    text = tf.strings.regex_replace(text, '[.?!,]', r' \0 ')
    # Strip whitespace.
    text = tf.strings.strip(text)

    text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
    return text

In [21]:
example_text = tf.constant('Ciao, tutto bene?')


In [22]:
print(example_text.numpy().decode())
print(tf_lower_and_split_punct(example_text).numpy().decode())

Ciao, tutto bene?
[START] ciao ,  tutto bene ? [END]


## Text Vectorization
for both input and target

In [24]:
max_vocab_size = 5000

input_text_processor = preprocessing.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_vocab_size)

In [25]:
input_text_processor.adapt(inp)

# Here are the first 10 words from the vocabulary:
input_text_processor.get_vocabulary()[:10]


['', '[UNK]', '[START]', '[END]', '.', 'tom', '?', 'non', 'e', 'di']

In [26]:
output_text_processor = preprocessing.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_vocab_size)

output_text_processor.adapt(targ)
output_text_processor.get_vocabulary()[:10]

['', '[UNK]', '[START]', '[END]', '.', 'you', 'tom', 'i', 'to', '?']

In [32]:
for x,y in dataset.take(1):
    print(x)
    print(y)

tf.Tensor(
[b'Penso di avere una linea di febbre.' b'Qualsiasi cosa va qui.'
 b'Ho fatto vedere a Tom come si fa.'
 b"La giustizia si trova in un'aula di giustizia."
 b'Vi aspetter\xc3\xb2 nella mia camera.'
 b'Tom \xc3\xa8 nato nello spazio.' b'\xc3\x88 la proprietaria.'
 b'Snocciolammo delle ciliegie per la torta.'
 b'Sono venuti tutti tranne Tom.' b'Io non me la ricordo.'
 b'Lei \xc3\xa8 complesso.' b'Non pu\xc3\xb2 pi\xc3\xb9 farlo.'
 b'Ho visto alcune persone nuotare nel fiume.'
 b'So che mi sta osservando.' b'Voi siete oberati di lavoro.'
 b'Ha una camera pi\xc3\xb9 economica?' b'Volevo venire.'
 b'Era di buonumore.' b'La forchetta cadde dal tavolo.'
 b'La camera \xc3\xa8 troppo grande.' b'Le sparer\xc3\xb2.'
 b"Lui fa bollire dell'acqua in una caffettiera."
 b'Ci hanno dato la loro parola.' b'Puoi metterla ovunque.'
 b'Lascia i libri e le riviste cos\xc3\xac come sono.'
 b'Alain ha concentrato tutte le sue attenzioni sulla sua gatta.'
 b"Tom ha bisogno di un po' di aiuto." b'Io 

In [33]:
example_tokens = input_text_processor(x)


In [35]:
print(x[0])
print(example_tokens[0])

tf.Tensor(b'Penso di avere una linea di febbre.', shape=(), dtype=string)
tf.Tensor([   2   59    9  178   23 3167    9 2032    4    3    0    0    0], shape=(13,), dtype=int64)


In [40]:
embedding_dim = 256
units = 1024


## Encoder

Pass the tokens, it learns some embeddings of those tokens and then pass them through a GRU RNN to process these vectors. It returns the sequence and the state

In [37]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, input_vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        self.enc_units = enc_units
        self.input_vocab_size = input_vocab_size

        # The embedding layer converts tokens to vectors
        self.embedding = tf.keras.layers.Embedding(self.input_vocab_size,
                                                   embedding_dim)

        # The GRU RNN layer processes those vectors sequentially.
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       # Return the sequence and state
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, tokens, state=None):
        #shape_checker = ShapeChecker()
        #shape_checker(tokens, ('batch', 's'))

        # 2. The embedding layer looks up the embedding for each token.
        vectors = self.embedding(tokens)
        #shape_checker(vectors, ('batch', 's', 'embed_dim'))

        # 3. The GRU processes the embedding sequence.
        #    output shape: (batch, s, enc_units)
        #    state shape: (batch, enc_units)
        output, state = self.gru(vectors, initial_state=state)
        #shape_checker(output, ('batch', 's', 'enc_units'))
        #shape_checker(state, ('batch', 'enc_units'))

        # 4. Returns the new sequence and its state.
        return output, state

In [42]:
# Convert the input text to tokens.
example_tokens = input_text_processor(x)

# Encode the input sequence.
encoder = Encoder(input_text_processor.vocabulary_size(),
                  embedding_dim, units)
example_enc_output, example_enc_state = encoder(example_tokens)

print(f'Input batch, shape (batch): {x.shape}')
print(f'Input batch tokens, shape (batch, s): {example_tokens.shape}')
print(f'Encoder output, shape (batch, s, units): {example_enc_output.shape}')
print(f'Encoder state, shape (batch, units): {example_enc_state.shape}')

Input batch, shape (batch): (64,)
Input batch tokens, shape (batch, s): (64, 13)
Encoder output, shape (batch, s, units): (64, 13, 1024)
Encoder state, shape (batch, units): (64, 1024)


## Attention

In [53]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        # For Eqn. (4), the  Bahdanau attention
        self.W1 = tf.keras.layers.Dense(units, use_bias=False)
        self.W2 = tf.keras.layers.Dense(units, use_bias=False)

        self.attention = tf.keras.layers.AdditiveAttention()

    def call(self, query, value, mask):
    
        # From Eqn. (4), `W1@ht`.
        w1_query = self.W1(query)

        # From Eqn. (4), `W2@hs`.
        w2_key = self.W2(value)

        query_mask = tf.ones(tf.shape(query)[:-1], dtype=bool)
        value_mask = mask

        context_vector, attention_weights = self.attention(
            inputs = [w1_query, value, w2_key],
            mask=[query_mask, value_mask],
            return_attention_scores = True,
        )


        return context_vector, attention_weights

## Decoder

In [50]:
class DecoderInput(typing.NamedTuple):
    new_tokens: Any
    enc_output: Any
    mask: Any

class DecoderOutput(typing.NamedTuple):
    logits: Any
    attention_weights: Any


class Decoder(tf.keras.layers.Layer):
    def __init__(self, output_vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.output_vocab_size = output_vocab_size
        self.embedding_dim = embedding_dim

        # For Step 1. The embedding layer convets token IDs to vectors
        self.embedding = tf.keras.layers.Embedding(self.output_vocab_size,
                                                   embedding_dim)

        # For Step 2. The RNN keeps track of what's been generated so far.
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

        # For step 3. The RNN output will be the query for the attention layer.
        self.attention = BahdanauAttention(self.dec_units)

        # For step 4. Eqn. (3): converting `ct` to `at`
        self.Wc = tf.keras.layers.Dense(dec_units, activation=tf.math.tanh,
                                        use_bias=False)

        # For step 5. This fully connected layer produces the logits for each
        # output token.
        self.fc = tf.keras.layers.Dense(self.output_vocab_size)
        
    def call(self,inputs: DecoderInput,state=None) -> Tuple[DecoderOutput, tf.Tensor]:


        # Step 1. Lookup the embeddings
        vectors = self.embedding(inputs.new_tokens)

        # Step 2. Process one step with the RNN
        rnn_output, state = self.gru(vectors, initial_state=state)


        # Step 3. Use the RNN output as the query for the attention over the
        # encoder output.
        context_vector, attention_weights = self.attention(
          query=rnn_output, value=inputs.enc_output, mask=inputs.mask)

        # Step 4. Eqn. (3): Join the context_vector and rnn_output
        #     [ct; ht] shape: (batch t, value_units + query_units)
        context_and_rnn_output = tf.concat([context_vector, rnn_output], axis=-1)

        # Step 4. Eqn. (3): `at = tanh(Wc@[ct; ht])`
        attention_vector = self.Wc(context_and_rnn_output)

        # Step 5. Generate logit predictions:
        logits = self.fc(attention_vector)

        return DecoderOutput(logits, attention_weights), state

        
        


In [54]:
decoder = Decoder(output_text_processor.vocabulary_size(),
                  embedding_dim, units)

In [56]:
# Convert the target sequence, and collect the "[START]" tokens
example_output_tokens = output_text_processor(y)

start_index = output_text_processor.get_vocabulary().index('[START]')
first_token = tf.constant([[start_index]] * example_output_tokens.shape[0])

In [59]:
dec_result, dec_state = decoder(
    inputs = DecoderInput(new_tokens=first_token,
                          enc_output=example_enc_output,
                          mask=(example_tokens != 0)),
    state = example_enc_state
)

print(f'logits shape: (batch_size, t, output_vocab_size) {dec_result.logits.shape}')
print(f'state shape: (batch_size, dec_units) {dec_state.shape}')

logits shape: (batch_size, t, output_vocab_size) (64, 1, 5000)
state shape: (batch_size, dec_units) (64, 1024)


## Training del modello

In [63]:
class MaskedLoss(tf.keras.losses.Loss):
    def __init__(self):
        self.name = 'masked_loss'
        self.loss = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction='none')

    def __call__(self, y_true, y_pred):
    
        # Calculate the loss for each item in the batch.
        loss = self.loss(y_true, y_pred)

        # Mask off the losses on padding.
        mask = tf.cast(y_true != 0, tf.float32)
        loss *= mask

        # Return the total.
        return tf.reduce_sum(loss)

In [62]:
class TrainTranslator(tf.keras.Model):
    def __init__(self, embedding_dim, units,
               input_text_processor,
               output_text_processor, 
               use_tf_function=True):
        super().__init__()
        # Build the encoder and decoder
        encoder = Encoder(input_text_processor.vocabulary_size(),
                          embedding_dim, units)
        decoder = Decoder(output_text_processor.vocabulary_size(),
                          embedding_dim, units)

        self.encoder = encoder
        self.decoder = decoder
        self.input_text_processor = input_text_processor
        self.output_text_processor = output_text_processor
        self.use_tf_function = use_tf_function

    def train_step(self, inputs):
        if self.use_tf_function:
            return self._tf_train_step(inputs)
        else:
            return self._train_step(inputs)

In [ ]:
### Finire qua